In [1]:
from utilities.postprocessing.postprocessing import * 
from utilities.postprocessing.main import *

%reload_ext autoreload
%autoreload 2

/home/francesco/.local/lib/python3.6/site-packages/skimage/novice/__init__.py:103: UserWarning: The `skimage.novice` module was deprecated in version 0.14. It will be removed in 0.16.
  warnings.warn("The `skimage.novice` module was deprecated in version 0.14. "


     Unnamed: 0             filename           map  height
0             0   slope_rocks3-1.0-0  slope_rocks3     1.0
1             1   slope_rocks3-1.0-1  slope_rocks3     1.0
2             2   slope_rocks3-1.0-2  slope_rocks3     1.0
3             3   slope_rocks3-1.0-3  slope_rocks3     1.0
4             4   slope_rocks3-1.0-4  slope_rocks3     1.0
5             5   slope_rocks3-1.0-5  slope_rocks3     1.0
6             6   slope_rocks3-1.0-6  slope_rocks3     1.0
7             7   slope_rocks3-1.0-7  slope_rocks3     1.0
8             8   slope_rocks3-1.0-8  slope_rocks3     1.0
9             9   slope_rocks3-1.0-9  slope_rocks3     1.0
10           10  slope_rocks3-1.0-10  slope_rocks3     1.0
11           11  slope_rocks3-1.0-11  slope_rocks3     1.0
12           12  slope_rocks3-1.0-12  slope_rocks3     1.0
13           13  slope_rocks3-1.0-13  slope_rocks3     1.0
14           14  slope_rocks3-1.0-14  slope_rocks3     1.0
15           15  slope_rocks3-1.0-15  slope_rocks3     1

In [2]:
from glob import glob

class TraversabilityDir():
    def __init__(self, root, maps_dir):
        self.root, self.maps_dir = root, maps_dir

    @property
    def meta(self):
        return pd.read_csv(self.bags_dir + '/meta.csv')

    @property
    def bags_dir(self):
        return path.normpath(self.root + '/bags')

    @property
    def csvs_dir(self):
        return path.normpath(self.root + '/csvs')

    @property
    def csvs_parsed_dir(self):
        return path.normpath(self.root + '/csvs_parsed')

    @property
    def csvs_patches_dir(self):
        return path.normpath(self.root + '/csvs_patches')

    @property
    def patches_dir(self):
        return path.normpath(self.root + '/patches')

    def should_convert2bag(self, filename):
        return path.exists(self.csvs + '/' + filename + '.csv')

    def exist(self, filename):
        return path.exists(self.csvs_parsed + '/' + filename + '.csv')
    

In [3]:
tr_dir = TraversabilityDir(root='/media/francesco/saetta/krock-dataset/val', maps_dir='./maps/val')

In [4]:
tr_dir.bags_dir

'/media/francesco/saetta/krock-dataset/val/bags'

In [14]:
from utilities.pipeline import MultiThreadWrapper

def hmread(path):
    hm = cv2.imread(path)
    hm = cv2.cvtColor(hm, cv2.COLOR_BGR2GRAY)
    return hm

class TraversabilityDataframe():
    def __init__(self, df, root):
        self.df = df
        self.root = root 
        
    @classmethod
    def from_dir(cls, dir, *args, **kwargs):
        return cls.from_meta(dir.meta, 
                             dir.csvs_patches_dir, 
                             dir.maps_dir, 
                             dir.bags_dir, 
                             dir.bags_parsed_dir,
                             dir.csvs_parsed_dir
                             *args, **kwargs)
    
        
    @classmethod
    def from_meta(cls, meta, 
                  df_dir, 
                  hm_dir, 
                  bags_dir=None, 
                  bags_parsed_dir=None,
                  csvs_parsed_dir=None
                  *args, **kwargs):
        names = meta['filename']
        maps = meta['map']
#         try:
#             df = pd.read_csv(df_dir + '/' + names[2] + '.csasdv')
#         except FileNotFoundError:
        hm = hmread(hm_dir + '/' + maps[2] + '.png')
        return cls.from_bag_path(path.normpath(bags_dir + '/' + names[2] + '.bag'), hm,  *args, **kwargs)

#         return MultiThreadWrapper(16, cls.from_bag, (names))
#         return cls.from_bag_path(path.normpath(bags_dir + '/' + names[0] + '.bag'), hm,  *args, **kwargs)
#         return cls(df, *args, **kwargs)

    @classmethod
    def from_bag_path(cls, bag_path, bags_parsed_dir, *args, **kwargs):
        df = rosbag_pandas.bag_to_dataframe(bag_path)
        
#         store
        return cls.from_bag_parsed(df, *args, **kwargs)
    
    @classmethod
    def from_bag_parsed(cls, df_path, hm, *args, **kwargs):
        df = pd.read_csv(df_path)
        df = ParseDataframe()(df)
        df = AddHMcoordinates(df),
        df = CleanDataframe(lower_bound=1, offset=22)(df, hm)
        df = drop_uselesss_columns(df)
        # get patches from bag converted 2 df
#         store
        return cls(df, '/')
    
    @classmethod
    def from_df(cls):
        # get patches from
#         store
        pass


In [15]:
TraversabilityDataframe.from_dir(tr_dir, 0.66, root='/').df

[INFO] something exploded while converting the ros time.


AttributeError: 'tuple' object has no attribute 'loc'

In [ ]:
meta.iloc[0]